# Fit the sinusoidal amplitudes using the RooComponentsPDF class

Here we use the RooComponentsPDF class to provide fast Simulated normalisation integration.
First load the fit classes.

In [1]:
import ROOT
ROOT.gROOT.ProcessLine(".x $BRUFIT/macros/LoadBru.C")
%jsroot

Welcome to JupyROOT 6.16/00


Use an instance of the FitManager class to organise the fit and give it an ouput directory.

In [2]:
fm = ROOT.FitManager()
fm.SetUp().SetOutDir("outObs2/");

Now load the experimentally measured variables (or observables). I must use the name of the variables in the tree.

Note the slighlty different notation for the Category PolState. If any events had a PolState of 0 for example, then they would not be included in the fit.

In [3]:
fm.SetUp().LoadVariable("Phi[-180,180]"); 
fm.SetUp().LoadVariable("Pol[0,1]"); 
fm.SetUp().LoadCategory("PolState[Polp=1,Pol0=0,Polm=-1]");

If I want to use any other variable, or example to apply a cut, I must load it as an AuxVar. The limits given will be applied to the datset. Here we will not apply it, but it is left commented out to show what you could do.

In [4]:
#fm.SetUp().LoadAuxVar("M1[0,10]"); //Load Aux Var, limits used as cut 
#fm.SetUp().AddCut("M1>2"); //Additional cut based on vars or aux vars

IMPORTANT here we set the event ID variable. Each event in your tree should have a unique id with which to synchronise with weights. This is useful as it allows us to break the data up, perform seperate fits then combine the weights afterwards. The weights are not written into the input tree, rather they are stored seperately (as HS::Weights) and combined when required. This allows you to use various sets of weights when performing fits or regenerate the weights after some corrections etc.

In [5]:
fm.SetUp().SetIDBranchName("fgID");

Create and load into the fit manager my PDF class. Phi, Pol and PolState all match the variables loaded above.

Here we define the functional parts seperately although they could be done on the fly on the FactoryPDF

In [6]:
fm.SetUp().LoadFormula("SIN2=@PolState[]*@Pol[]*sin(2*(@Phi[])/57.29578)");
fm.SetUp().LoadFormula("COS2=@PolState[]*@Pol[]*cos(2*(@Phi[])/57.29578)");

[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooFormulaVar::SIN2
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooFormulaVar::COS2


Here we load the fit parameters on the fly in the factory PDF e.g. A[0,-1,1] initial value 0 and range between -1 and 1. If A appears multiple times I should only include the [0,-1,1] in the first instance and then just A from then on.

WE could have Loaded them explicitly first. This may help reading in more complicate functions. e.g.

    fm.SetUp().LoadParameter("A[0,-1,1]");
    
Then just use A rather than A[0,-1,1] in FactoryPDF


    1 + A*COS2 + B*SIN2 (first argument = the 1 )
    
Note : seperates different products = +
     ; seperates different terms in the product = *
     
Note the 1 in "SigAsym(1" corresponds to the 1 + in the formula. This may also be set to 0 if the formula does not have a 1+ at the start.

In [7]:
fm.SetUp().FactoryPDF("RooComponentsPDF::SigAsym(1,{Phi,Pol,PolState},=A[0,-1,1];COS2:B[0,-1,1];SIN2)");
fm.SetUp().LoadSpeciesPDF("SigAsym",1);
fm.SetUp().WS().Print("v")

[#1] INFO:ObjectHandling -- RooWorkspace::import() importing HS::FIT::RooComponentsPDF::SigAsym

RooWorkspace()  contents

variables
---------
(A,B,Phi,Pol,PolState,Yld_SigAsym,fgID)

p.d.f.s
-------
HS::FIT::RooComponentsPDF::SigAsym[ AllObservables=(Phi,Pol) AllCategories=(PolState) AllComponents=(A,COS2,B,SIN2) Phi=Phi Pol=Pol PolState=PolState A=A COS2=COS2 B=B SIN2=SIN2 ] = 1

functions
--------
RooFormulaVar::COS2[ actualVars=(PolState,Pol,Phi) formula="PolState*Pol*cos(2*(Phi)/57.29578)" ] = 0.5
RooFormulaVar::SIN2[ actualVars=(PolState,Pol,Phi) formula="PolState*Pol*sin(2*(Phi)/57.29578)" ] = 0



Now I load my data, here I will use both data to fit and MC data for the integral calculation. Note in the latter case the string "SigAsym" matches the name given to the PDF object above. In principal I could fit multiple PDFs and give them each different MC data.

The strings are treename, filename and PDF name (simulation only).

In [8]:
fm.LoadData("MyModel","Data.root");
fm.LoadSimulated("MyModel","MC.root","SigAsym");

DataEvents::Load MyModel with 1 files


Now I attach the weights from my sPlot fit. I want to use the signal weights which were given the name "Signal" in the sPlot notebook. 

Also there are two possible sets of weights from using a Gaussian or simulated PDF. You can switch between either here.

In [9]:
fm.Data().LoadWeights("Signal","outSplot/Tweights.root");
#fm.Data().LoadWeights("Signal","outSplotSim/Tweights.root"); #use weights from simulated PDF

Weights HSsWeights contains 100000 events associated file is 
ID branch name : fgID
Species are : 
BG
Signal
The first ten entries are :
0 0.160932 0.839068 
1 -0.147928 1.14793 
2 1.15202 -0.15202 
3 1.43807 -0.438065 
4 1.42256 -0.422553 
5 -0.008293 1.00829 
6 1.4532 -0.453194 
7 1.44715 -0.447147 
8 0.399246 0.600753 
9 1.44954 -0.449542 
  DataEvents::LoadWeights using Signal weights /work/Dropbox/HaSpect/dev/brufit/tutorials/WeightedObervable/outSplot/Tweights.root HSsWeights


I can provide additional options to speed up the fit. Here I use 3 CPUs which parallelises the likelihood calculation.

And I set optimise to 1 which calculates and caches formulas in the first interation, then uses the cached values in future iterations of the likelihood calculation. This is well matched to the ComponentsPDF .

In [10]:
fm.SetUp().AddFitOption(ROOT.RooFit.NumCPU(3));
fm.SetUp().AddFitOption(ROOT.RooFit.Optimize(1));

Now run the fits. I use the Process classes which allow me to choose between running directly here on a single core or multicore via PROOF-lite. It doesn't make sense to run with PROOF unless multiple splits have been defined with LoadBinVar or you are using Bootstrap, in which case you should relate the number of cores requested to the number of splits or bootstraps.

** For me the programme hangs at this cell once the fit is done and the plots created. If I stop the cell I can see the plots and they are already saved into the results file.

In [11]:
ROOT.Here.Go(fm)

KeyboardInterrupt: 


[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooFormulaVar::SIN2
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing RooFormulaVar::COS2
[#1] INFO:ObjectHandling -- RooWorkspace::import() importing HS::FIT::RooComponentsPDF::SigAsym
 RooAbsData& DataEvents::Get  /work/Dropbox/HaSpect/dev/brufit/tutorials/WeightedObervable/Data.root tree MyModel weights 0x557da36e9020 Signal
FiledTree::~FiledTree()  tree name MyModel 100000 /work/Dropbox/HaSpect/dev/brufit/tutorials/WeightedObervable/Data.root
[#1] INFO:DataHandling -- RooAbsCategory::attachToTree(PolState) TTree branch PolState will be interpreted as category index
FiledTree::~FiledTree()  tree name MyModel 100000 /work/Dropbox/HaSpect/dev/brufit/tutorials/WeightedObervable/outObs2///DataInWeightedTree0.root
RooDataSet::DataEvents[Phi,Pol,PolState,fgID,weight:Signal] = 100000 entries (50038.5 weighted)
RooHSEventsPDF::SetEvTree SigAsym
RooHSEventsPDF::SetEvTree Ammended cut (PolState==1||PolState==0||PolState=

Info in <HS::FIT::RooComponentsPDF::RooHSEventsPDF::SetEvTree>:  with name MyModel and cut (PolState==1||PolState==0||PolState==-1)
Warning in <HS::FIT::RooComponentsPDF::RooHSEventsPDF::SetEvTree>: Branch Pol not found
Error in <TString::Replace>: first argument out of bounds: pos = -1, Length = 40
Error in <TString::Replace>: first argument out of bounds: pos = -1, Length = 40
Warning in <HS::FIT::RooComponentsPDF::RooHSEventsPDF::SetEvTree>: Branch PolState not found
Info in <HS::FIT::RooComponentsPDF::RooHSEventsPDF::AddProtoData>: Added data branch Pol
Info in <HS::FIT::RooComponentsPDF::RooHSEventsPDF::AddProtoData>: Added data branch PolState
Info in Minuit2Minimizer::Hesse : Hesse is valid - matrix is accurate
Info in Minuit2Minimizer::Hesse : Hesse is valid - matrix is accurate


All plots and fit parameters will be saved into the output directory Results.root file. If LoadBinVar splits were applied then they will be in directories related to the bin name.

If the plot does not appear you can try removing the %jsroot in the first cell. Unfortunately this will mean the histograms will not be interactive.